# Ceneo Scraper

## Components Of Single Opinion


|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|["span.user-post__author-recomendation > em|recomendation]|author|
|author’s recommendation|["span.user-post__score-count"]|recomendation|
|score expressed in number of stars|["div.user-post__score-count"]|score|
|opinion’s content|["div.user-post__text"]|content|
|list of product advantages|["div.review.feture__title--positives ~ div.review-feture__item"]|pros|
|list of product disadvantages|["div.review.feture__title--negatives ~ div.review-feture__item"]|cons|
|how many users think that opinion was helpful|button.vote-yes > span|helpful|
|how many users think that opinion was unhelpful|button.vote-no > span|unhelpful|
|publishing date|span.user-post__published > time:nth-child(1)|publish_date|
|purchase date|span.user-post__published > time:nth-child(2)|purchase_date|


## Structure of single opinion

In [ ]:
selectors = {
    "opinion_id" : [None, "data-entry-id"],
    "author" : ["span.user-post__author-recomendation"],
    "recomendation" : ["span.user-post__score-count"],
    "rating" : ["div.user-post__score-count"],
    "content" : ["div.user-post__text"],
    "pros" : ["div.review.feture__title--positives ~ div.review-feture__item", None, True],
    "cons" : ["div.revuew.fetere__title--negatives ~ div.review-feture__item", None, True],
    "helpful_count" : ["button.vote-yes > span"],
    "unhelpful_count" : ["button.vote-no > span"],
    "publish_date" : ["span.user-post__published > time:nth-child(1)", "datetime"],
    "purchase_date" : ["span.user-post__published > time:nth-child(2)","datetime"],
}

# Loading libraries

In [ ]:
import requests
from bs4 import BeautifulSoup

## Function to extract data from html code

In [ ]:
def extract(ancestor, selector, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute] for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute]
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().stip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.get_text().strip()

# Sending request to Ceneo.pl server

In [ ]:
product_id = "41586775"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)
response.status_code

## Extracting opinions from HTML code

In [ ]:
page_dom = BeautifulSoup(response.text, "html.parser")
opinions = page_dom.select("div.js_product-review")
opinion = page_dom.select_one("div.js_product-review")

print(type(page_dom))
print(type(opinion))
print(type(opinions))
print(opinion)

## Extracting Components for single opinion

In [ ]:
single_opinion = {
    key: extract(opinion, *value)
        for key, value in selectors.items()
}